Pattern matching is a powerful tool to process records structured like nested mappings and sequences, which we often need to read from JSON APIs and databases with semi-structured schemas, like MongoDB, EdgeDB, or PostgreSQL.  (https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/ch03.html#idm46582495648864_)

This is beacuse it can work with any mappings from the collections library, not just dicts. The order of the items in the dict/list of mappings also does not matter
Also having extra information does not change the match, but you can capture it with **, as you can see in the baseball record case and call below:

In [ ]:

def get_team(record:dict) -> list:
    match record:
        case {'sport': 'baseball', 'api': 'mlb', 'names': [*names], **baseball_details}:
            print(f"Baseball details: {baseball_details}")
            return names
        case {'sport': 'hockey', 'api': 'nhl', 'name': name}:
            return [name]
        case {'sport': 'hockey'}:
            raise ValueError(f"Invalid 'hockey' record: {record!r}")
        case {'sport': 'wrestling', 'wrestler': name}:
            return [name]
        case _:
            raise ValueError(f'Invalid record: {record!r}')

In [ ]:
from collections import OrderedDict

# Test datasets
baseball_record = {'sport': 'baseball', 'api': 'mlb', 'names': ['John', 'Alice', 'Mike']}
hockey_record = {'sport': 'hockey', 'api': 'nhl', 'name': 'Emma'}
invalid_hockey_record = {'sport': 'hockey', 'api': 'nhl'}
wrestling_record = {'sport': 'wrestling', 'wrestler': 'Tom'}
invalid_record = {'sport': 'football', 'team': 'Manchester United'}
spam_record = 'spam, spam, spam'

# Test with the baseball record
print(get_team(baseball_record))

# Test with the hockey record
print(get_team(hockey_record))

# # Test with the invalid hockey record
# print(get_team(invalid_hockey_record))

# Test with the wrestling record
print(get_team(wrestling_record))

# # Test with the invalid record
# print(get_team(invalid_record))

# Test with the spam record
# print(get_team(spam_record))

ordered_dict_record = OrderedDict([
    ('extra_field', 'Extra Field Value'),
    ('api', 'mlb'),
    ('sport', 'baseball'),
    ('names', ['Alice', 'John', 'Mike']),
    ('extra_field_2', 'Another Extra Field')
])

# Test with the ordered dict record
print(get_team(ordered_dict_record))


## Some different ways to capture data from class instances using pattern matching.

In [18]:
import typing
from datetime import datetime


class Log(typing.NamedTuple):
    timestamp: datetime
    ip: str
    crudop: str
    url: str
    status_code: int
    browser: str
    transer_bytes: int
    duration: float

log_data = [
    Log("2023-06-30 10:25:12", "192.0.2.42", "GET", "/index.html", 200, "Mozilla/5.0", 1256, 0.125),
    Log("2023-06-30 10:26:07", "192.0.2.53", "POST", "/form_submit", 302, "Mozilla/5.0", 732, 0.256),
    Log("2023-06-30 10:27:34", "192.0.2.28", "GET", "/favicon.ico", 404, "Mozilla/5.0", 0, 0.010),
    Log("2023-06-30 10:30:29", "192.0.2.42", "GET", "/img/logo.png", 200, "Mozilla/5.0", 2048, 0.175),
    Log("2023-06-30 10:32:45", "192.0.2.62", "POST", "/data_upload", 201, "Mozilla/5.0", 5000, 2.345),
    Log("2023-06-30 10:34:12", "192.0.2.75", "GET", "/big_file.txt", 200, "Mozilla/5.0", 10000, 5.623),
    Log("2023-06-30 10:36:47", "192.0.2.42", "POST", "/data_processing", 202, "Mozilla/5.0", 1500, 10.789),
    Log("2023-06-30 10:40:29", "192.0.2.85", "GET", "/another_big_file.txt", 200, "Mozilla/5.0", 12000, 6.129),
    Log("2023-06-30 10:42:18", "192.0.2.92", "GET", "/slow_page.html", 200, "Mozilla/5.0", 2500, 12.356),
]




## This will match on crud operation based on a class keyword and also ip and time of initial request.  Keyword matches are usually more readable:

In [25]:
def match_post_keyword():
    results = []
    for log in log_data:
        match log:
            case Log(crudop='POST', status_code=post_code, timestamp=ts):
                results.append((ts, post_code))
    return results

# The walrus operator := allows you to assign values to variables as part of an expression.

print(results := match_post_keyword())

[('2023-06-30 10:26:07', 302), ('2023-06-30 10:32:45', 201), ('2023-06-30 10:36:47', 202)]


## You can also do this with positional patterns, not using class keywords

In [26]:
def match_post_positional():    
    results = []
    for log in log_data:
        match log:
            case Log(timestamp, _, 'POST', _, post_code, _, _, _):
                results.append((timestamp, post_code))
    return results

print(results := match_post_positional())

[('2023-06-30 10:26:07', 302), ('2023-06-30 10:32:45', 201), ('2023-06-30 10:36:47', 202)]


## This will match on request durations longer than 2 seconds based on class keyword and also capture the crud operation (crudop) and time of initial request:

In [27]:
def match_slow_requests():
    results = []
    for log in log_data:
        match log:
            case Log(crudop=slowcrud, timestamp=slowtime, duration=d) if d > 2:
                results.append((slowcrud, slowtime, d))
    return results

print(results := match_slow_requests())

[('POST', '2023-06-30 10:32:45', 2.345), ('GET', '2023-06-30 10:34:12', 5.623), ('POST', '2023-06-30 10:36:47', 10.789), ('GET', '2023-06-30 10:40:29', 6.129), ('GET', '2023-06-30 10:42:18', 12.356)]


Python 3.10 introduced structural pattern matching via the `match` / `case` syntax, providing a powerful tool to inspect and handle data more expressively and readably. Here are three examples of how it can be used:

1. **Basic pattern matching**: The most basic form of pattern matching is matching literals or variable binding:

    ```python
    def http_response_status(code):
        match code:
            case 200:
                return "OK"
            case 404:
                return "Not Found"
            case _:
                return "Unknown"

    print(http_response_status(200))  # outputs: "OK"
    ```

    In this example, we're matching the `code` with a few literal values (200, 404) and we use `_` as a catch-all case.

2. **Sequence matching**: You can match sequences like lists or tuples, and even use patterns for the items in the sequences:

    ```python
    def describe_point(point):
        match point:
            case (0, 0):
                return "Origin"
            case (0, y):
                return f"Y={y}"
            case (x, 0):
                return f"X={x}"
            case (x, y):
                return f"X={x}, Y={y}"
            case _:
                raise ValueError("Invalid point")

    print(describe_point((10, 0)))  # outputs: "X=10"
    ```

    In this example, we're matching the `point` tuple with various patterns. The `(0, y)` pattern for instance matches a tuple where the first element is 0, and binds the second element to `y`.

3. **Class matching**: This form is very powerful when working with classes:

    ```python
    class Circle:
        def __init__(self, radius):
            self.radius = radius

    class Rectangle:
        def __init__(self, width, height):
            self.width = width
            self.height = height

    def area(shape):
        match shape:
            case Circle(r=radius):
                return 3.14 * radius ** 2
            case Rectangle(w=width, h=height):
                return width * height
            case _:
                raise TypeError("Invalid shape")

    c = Circle(1)
    r = Rectangle(2, 3)
    print(area(c))  # outputs: 3.14
    print(area(r))  # outputs: 6
    ```

    In this example, we're matching the `shape` object against the Circle and Rectangle classes. The patterns `Circle(r=radius)` and `Rectangle(w=width, h=height)` don't just match the class, but also bind the specified attribute to a variable.

This is just scratching the surface, pattern matching in Python 3.10 also supports more complex patterns such as or-patterns, matching on specific attribute values, matching on types, and more.


Sure, here are two examples for each of sequence matching and class matching.

**Sequence matching:**

1. Matching patterns within a list:

    ```python
    def process_data(data):
        match data:
            case []:
                return "No data"
            case [first_item]:
                return f"One item: {first_item}"
            case [first_item, second_item]:
                return f"Two items: {first_item}, {second_item}"
            case _:
                return "More than two items"

    print(process_data([1, 2, 3]))  # outputs: "More than two items"
    ```

    In this example, we're matching the `data` list with various patterns, including an empty list, a list with one item, and a list with two items.

2. Matching nested sequences:

    ```python
    def identify_shape(shape):
        match shape:
            case ['circle', _]:
                return "It's a circle."
            case ['rectangle', width, height]:
                return f"It's a rectangle of width {width} and height {height}."
            case _:
                return "Unknown shape."

    print(identify_shape(['rectangle', 5, 10]))  # outputs: "It's a rectangle of width 5 and height 10."
    ```

    Here, we're matching a list that describes a shape. The first element of the list is a string describing the shape, and the remaining elements are parameters of the shape.

**Class matching:**

1. Matching instances of a specific class:

    ```python
    class Dog:
        def __init__(self, name):
            self.name = name

    class Cat:
        def __init__(self, name):
            self.name = name

    def identify_pet(pet):
        match pet:
            case Dog(name=name):
                return f"It's a dog named {name}"
            case Cat(name=name):
                return f"It's a cat named {name}"
            case _:
                return "Unknown pet"

    d = Dog('Rex')
    c = Cat('Whiskers')
    print(identify_pet(d))  # outputs: "It's a dog named Rex"
    print(identify_pet(c))  # outputs: "It's a cat named Whiskers"
    ```

    In this example, we're matching the `pet` object against the Dog and Cat classes, and extracting the name of the pet.

2. Matching on specific attribute values:

    ```python
    class Car:
        def __init__(self, brand, electric):
            self.brand = brand
            self.electric = electric

    def describe_car(car):
        match car:
            case Car(brand="Tesla", electric=True):
                return "It's a Tesla electric car."
            case Car(brand=brand, electric=False):
                return f"It's a {brand}, but not electric."
            case _:
                return "Unknown car"

    c1 = Car('Tesla', True)
    c2 = Car('Ford', False)
    print(describe_car(c1))  # outputs: "It's a Tesla electric car."
    print(describe_car(c2))  # outputs: "It's a Ford, but not electric."
    ```

    Here, we're matching the `car` object with specific attribute values. For example, the pattern `Car(brand="Tesla", electric=True)` matches a Car where `brand` is 'Tesla' and `electric` is True.
